In [7]:
import re
from itertools import product, combinations_with_replacement
from operator import itemgetter, add, mul, sub, truediv, pow
from sympy import srepr, sympify, symbols, exp, sin, cos, ln, asin, acos
x,c = symbols('x c')

In [2]:
all_nodes = {
    'x': {
        'children': 0,
        'op': x
    },
    'c': {
        'children': 0,
        'op': c # Konstante
    },
    'exp': {
        'children': 1,
        'op': exp
    },
    'ln': {
        'children': 1,
        'op': ln
    },
    'sin': {
        'children': 1,
        'op': sin
    },
    'cos': {
        'children': 1,
        'op': cos
    },
    'asin': {
        'children': 1,
        'op': asin
    },
    'acos': {
        'children': 1,
        'op': acos
    },
    '+': {
        'children': 2,
        'op': add,
        'commutative': True
    },
    '-': {
        'children': 2,
        'op': sub,
        'commutative': False
    },
    '*': {
        'children': 2,
        'op': mul,
        'commutative': True
    },
    '/': {
        'children': 2,
        'op': truediv,
        'commutative': False
    },
    # '**': {
    #     'children': 2,
    #     'op': pow,
    #     'commutative': False
    # }
}

In [4]:
def generate_expressions(sub_expressions):
    for properties in all_nodes.values():
        children, operation = itemgetter('children', 'op')(properties)
        if children == 2 and properties['commutative'] == True:
            for left, right in combinations_with_replacement(sub_expressions, 2):
                left, right = sympify(left), sympify(right)
                if (left == c and right == c) or (operation == add and left == right):
                    pass
                else:
                    yield operation(left, right)
        elif children == 2:
            for left, right in product(sub_expressions, repeat=2):
                left, right = sympify(left), sympify(right)
                if left != right:
                    yield operation(left, right)
        elif children == 1:
            for expr in sub_expressions:
                expr = sympify(expr)
                if expr != c:
                    yield operation(expr)

In [8]:
def save_expressions(depth):
    uniques = set([x, c])
    if depth > 1:
        with open(f'uniques_ext_depth{depth - 1}.csv', 'r') as file:
            for line in file:
                uniques.add(line.strip())

    with open(f'expressions_ext_depth{depth}.csv', 'w') as file:
        for expr in generate_expressions(uniques):
            expr_str = str(expr)
            if 'x' in expr_str:
                file.write(str(expr))
                file.write('\n')
        for expr in uniques:
            file.write(str(expr))
            file.write('\n')

In [12]:
def cleanup_expressions(depth):
    uniques = set()
    with open(f'expressions_ext_depth{depth}.csv', 'r') as file:
        for line in file:
            # Filter out multiplications with an integer (always replacable by multiplication with c) and any c**2 (replacable with c)
            if not (re.search('(?<!\*)\*?\d\*(x|c)', line) or 'c**2' in line):
                uniques.add(line)
    with open(f'uniques_ext_depth{depth}.csv', 'w') as file:
        for line in uniques:
            file.write(line)

In [14]:
depth = 2
save_expressions(depth)
cleanup_expressions(depth)

In [16]:
cleanup_expressions(3)

: 